This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/abtesting).

## A/B Testing

A/B testing is a method that provides the ability to test out ML models for performance, accuracy or other useful benchmarks.  A/B testing is contrasted with the Wallaroo Shadow Deployment feature.  In both cases, two sets of models are added to a pipeline step:

* Control or Champion model:  The model currently used for inferences.
* Challenger model(s): One or more models that are to be compared to the champion model.

The two feature are different in this way:

| Feature | Description |
|---|---|
| A/B Testing | A subset of inferences are submitted to either the champion ML model or a challenger ML model. |
| Shadow Deploy | All inferences are submitted to the champion model and one or more challenger models. |

So to repeat:  A/B testing submits *some* of the inference requests to the champion model, some to the challenger model with one set of outputs, while shadow testing submits *all* of the inference requests to champion and shadow models, and has separate outputs.

This tutorial demonstrate how to conduct A/B testing in Wallaroo.  For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model and a challenger model.
* Create a pipeline that can ingest our submitted data with the champion model and the challenger model set into a A/B step.
* Run a series of sample inferences to display inferences that are run through the champion model versus the challenger model, then determine which is more efficient.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Steps

### Import libraries

Here we will import the libraries needed for this notebook.

In [15]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import pandas as pd
import json
from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [16]:
wallaroo.__version__

'2023.1.0'

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [27]:
import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

if "ARROW_ENABLED" not in os.environ or os.environ["ARROW_ENABLED"].casefold() == "False".casefold():
    arrowEnabled = False
else:
    arrowEnabled = True
print(arrowEnabled)

True


### Connect to the Wallaroo Instance

This command will be used to set up a connection to the Wallaroo cluster and allow creating and use of Wallaroo inference engines.

In [18]:
# Client connection from local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR SUFFIX"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace for all other commands.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [19]:
workspace_name = 'abhousetesting'

In [20]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

In [21]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'abhousetesting', 'id': 149, 'archived': False, 'created_by': '138bd7e6-4dc8-4dc1-a760-c9e721ef3c37', 'created_at': '2023-03-10T19:47:57.006484+00:00', 'models': [], 'pipelines': []}

### Set Up the Champion and Challenger Models

Now we upload the Champion and Challenger models to our workspace.  We will use two models:

1. `aloha-cnn-lstm` model.
2. `aloha-cnn-lstm-new` (a retrained version)

### Set the Champion Model

We upload our champion model, labeled as `control`.

In [41]:
#control =  wl.upload_model("aloha-control",   'models/aloha-cnn-lstm.zip').configure('tensorflow')
control = wl.upload_model("houseprice-control",'models/housing_control.zip').configure('tensorflow')

### Set the Challenger Model

Now we upload the Challenger model, labeled as `challenger`.

In [23]:
#challenger = wl.upload_model("aloha-challenger",   'models/aloha-cnn-lstm-new.zip').configure('tensorflow')
challenger = wl.upload_model("houseprice-challenger",'models/housing_challenger.zip').configure('tensorflow')

### Define The Pipeline

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

In [42]:
pipeline = (wl.build_pipeline("randomsplitpipeline-demo")
            .add_random_split([(2, control), (1, challenger)], "session_id"))

In [31]:
# for d in wl.list_deployments(): d.undeploy()

### Deploy the pipeline

Now we deploy the pipeline so we can run our inference through it.

In [43]:
experiment_pipeline = pipeline.deploy()

In [33]:
experiment_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.1.13',
   'name': 'engine-578f4f8694-x9f6k',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'randomsplitpipeline-demo',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'houseprice-challenger',
      'version': '9332b9fb-ad9e-4569-9f48-fa25a54a5f7e',
      'sha': '0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.1.14',
   'name': 'engine-lb-86bc6bd77b-brv44',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

# Run a single inference
Now we have our deployment set up let's run a single inference. In the results we will be able to see the inference results as well as which model the inference went to under model_id.  We'll run the inference request 5 times, with the odds are that the challenger model being run at least once.

In [48]:
# results = []
# if arrowEnabled is True:
#     # use dataframe JSON files
#     for x in range(5):
#         result = experiment_pipeline.infer_from_file("data/data-1.df.json")
#         display(result.loc[:,["out._model_split", "out.main"]])    
# else:
#     # use Wallaroo JSON files
#     results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
#     results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
#     results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
#     results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
#     results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
#     for result in results:
#         print(result[0].model())
#         print(result[0].data())
results = []
if arrowEnabled is True:
    # use dataframe JSON files
    for x in range(5):
        result = experiment_pipeline.infer(pd.DataFrame.from_records({"dense_16_input":{"0":[0.02675675,0.0,0.02677953,0.0,0.0010046,0.00951931,0.14795322,0.0027145,0.03550877,0.98536841,0.02988655,0.04031725,0.04298041]}}))
        # display(result)
        display(result.loc[:,["out._model_split", "out.dense_19"]])    
else:
    # use Wallaroo JSON files
    results.append(experiment_pipeline.infer({"dense_16_input":[[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145,  0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]]}))
    results.append(experiment_pipeline.infer({"dense_16_input":[[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145,  0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]]}))
    results.append(experiment_pipeline.infer({"dense_16_input":[[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145,  0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]]}))
    results.append(experiment_pipeline.infer({"dense_16_input":[[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145,  0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]]}))
    results.append(experiment_pipeline.infer({"dense_16_input":[[0.02675675, 0.0, 0.02677953, 0.0, 0.0010046, 0.00951931, 0.14795322, 0.0027145,  0.03550877, 0.98536841, 0.02988655, 0.04031725, 0.04298041]]}))
    for result in results:
        print(result[0].model())
        print(result[0].data())

,out._model_split,out.dense_19
0,"[{""name"":""houseprice-challenger"",""version"":""9332b9fb-ad9e-4569-9f48-fa25a54a5f7e"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[10.349835]


,out._model_split,out.dense_19
0,"[{""name"":""houseprice-challenger"",""version"":""9332b9fb-ad9e-4569-9f48-fa25a54a5f7e"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[10.349835]


,out._model_split,out.dense_19
0,"[{""name"":""houseprice-control"",""version"":""03f649d4-d5d7-4cba-88fe-a1be90f5a917"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[10.349835]


,out._model_split,out.dense_19
0,"[{""name"":""houseprice-control"",""version"":""03f649d4-d5d7-4cba-88fe-a1be90f5a917"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[10.349835]


,out._model_split,out.dense_19
0,"[{""name"":""houseprice-control"",""version"":""03f649d4-d5d7-4cba-88fe-a1be90f5a917"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[10.349835]


### Run Inference Batch

We will submit 1000 rows of test data through the pipeline, then loop through the responses and display which model each inference was performed in.  The results between the control and challenger should be approximately 2:1.

In [49]:
responses = []
if arrowEnabled is True:
    test_data = pd.read_json('./data/test_data_anomaly_df.json', orient="records")
    responses_anomaly = pd.DataFrame()
    # For the first 400 rows, submit that row as a separate DataFrame
    # Add the results to the responses_anomaly dataframe
    # for index, row in test_data.head(400).iterrows():
    # responses = pd.DataFrame()
    #Read in the test data as one dataframe
    # test_data = pd.read_json('data/test_data_anomaly_df.json')
    # For each row, submit that row as a separate dataframe
    # Add the results to the responses array
    for index, row in test_data.head(400).iterrows():
        responses.append(experiment_pipeline.infer(row.to_frame('dense_16_input').reset_index()))
        # display(responses)
    #now get our responses for each row
    # each r is a dataframe, then get the result from out.split into json and get the model name
    # for r in responses:
    #     display(r.loc[0]["out.split"])
    l = [json.loads(r.loc[0]["out._model_split"][0])["name"] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())
else:
    l = []
    responses =[]
    from data import test_data
    for nth in range(1000):
        responses.extend(experiment_pipeline.infer(test_data.data[nth]))
    l = [r.raw['model_name'] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())

houseprice-control       263
houseprice-challenger    137
Name: model, dtype: int64

### Test Challenger

Now we have run a large amount of data we can compare the results.

For this experiment we are looking for a significant change in the fraction of inferences that predicted a probability of the seventh category being high than 0.5 so we can determine whether our challenger model is more "successful" than the champion model at identifying category 7.

In [50]:
control_count = 0
challenger_count = 0

control_success = 0
challenger_success = 0

if arrowEnabled is True:
    # do nothing
    for r in responses:
        if json.loads(r.loc[0]["out._model_split"][0])["name"] == "houseprice-control":
            control_count += 1
            if(r.loc[0]["out.dense_19"][0] > .5):
                control_success += 1
        else:
            challenger_count += 1
            if(r.loc[0]["out.dense_19"][0] > .5):
               challenger_success += 1
else:
    for r in responses:
        if r.raw['model_name'] == "aloha-control":
            control_count += 1
            if(r.raw['outputs'][7]['Float']['data'][0] > .5):
                control_success += 1
        else:
            challenger_count +=1
            if(r.raw['outputs'][7]['Float']['data'][0] > .5):
                challenger_success += 1

print("control class 7 prediction rate: " + str(control_success/control_count))
print("challenger class 7 prediction rate: " + str(challenger_success/challenger_count))

control class 7 prediction rate: 1.0
challenger class 7 prediction rate: 1.0


### Logs

Logs can be viewed with the Pipeline method `logs()`.  For this example, only the first 5 logs will be shown.  For Arrow enabled environments, the model type can be found in the column `out._model_split`.

In [53]:
logs = experiment_pipeline.logs(limit=5)

if arrowEnabled is True:
    # display(logs)
    display(logs.loc[:,['time', 'out._model_split', 'out.dense_19']])
else:
    display(logs)

,time,out._model_split,out.dense_19
0,2023-03-10 20:02:52.438,"[{""name"":""houseprice-challenger"",""version"":""78d2b8fc-bf8c-4457-b620-c819c0b5de59"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[16.037834]
1,2023-03-10 20:02:52.897,"[{""name"":""houseprice-challenger"",""version"":""78d2b8fc-bf8c-4457-b620-c819c0b5de59"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[31.820295]
2,2023-03-10 20:17:51.424,"[{""name"":""houseprice-control"",""version"":""03f649d4-d5d7-4cba-88fe-a1be90f5a917"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[23.26468]
3,2023-03-10 20:17:51.758,"[{""name"":""houseprice-control"",""version"":""03f649d4-d5d7-4cba-88fe-a1be90f5a917"",""sha"":""0d03d638f7100827288c808c2cc6e5dae127f5f31e7f0f97ef70aa2078f11cc1""}]",[11.928354]


### Undeploy Pipeline

With the testing complete, we undeploy the pipeline to return the resources back to the environment.

In [54]:
experiment_pipeline.undeploy()

name,randomsplitpipeline-demo
created,2023-03-10 19:48:03.288037+00:00
last_updated,2023-03-10 20:04:06.158956+00:00
deployed,False
tags,
versions,"0b637d0d-423f-4507-858a-a1fcc6081a4a, da6de99c-e228-4338-961a-134a4a476f71, ddde66e5-8abf-426e-8c09-ca030c277e38, bf03a0f9-ba6b-4415-ac9a-7f69c2e4b276, 75abdcb8-d63d-48a0-83a3-6ab69865b385"
steps,houseprice-challenger
